In [13]:
import pandas as pd
import csv
import re
from nltk.wsd import lesk
from nltk.metrics import jaccard_distance
from nltk.corpus import stopwords
from scipy.stats import pearsonr
#from nltk.stem import WordNetLemmatizer
#from nltk.tokenize import WhitespaceTokenizer
#from nltk.metrics import jaccard_distance
#from scipy.stats import pearsonr
#from nltk.corpus import wordnet as wn
from nltk import pos_tag, word_tokenize

In [2]:
trial_path    = 'trial/STS.input.txt'
trial_gs_path = 'trial/STS.gs.txt'
trial_df      = pd.read_csv(trial_path,    sep='\t', lineterminator='\n', names=['sentence0','sentence1'], header=None, quoting=csv.QUOTE_NONE)
trial_gs      = pd.read_csv(trial_gs_path, sep='\t', lineterminator='\n', names=['labels'],                header=None, quoting=csv.QUOTE_NONE)

print(trial_df.shape, trial_gs.shape)

(6, 2) (6, 1)


In [21]:
length = trial_df.shape[0]
sims = []

for i in range(length):
    semantics = []

    for column in trial_df:
        senten   = trial_df[column][i]
        tokens   = word_tokenize(senten)
        tagged   = pos_tag(tokens)
        semantic = set()

        for idx, token in enumerate(tokens):
            context = [i for i in tokens if i != token]
            tag     = tagged[idx][1][0].lower()
            synset  = lesk(context, token.lower(), tag)

            #print(token, synset, tag)
            if synset:
                semantic.add(synset) 
            else:
                semantic.add(token.lower())

        #print(senten)
        #print(semantic)
        semantics.append(semantic)
        
    sims.append(1 - jaccard_distance(semantics[0], semantics[1]))
    print('similarity [0-1]:', sims[-1])

similarity [0-1]: 0.33333333333333337
similarity [0-1]: 0.33333333333333337
similarity [0-1]: 0.5714285714285714
similarity [0-1]: 0.33333333333333337
similarity [0-1]: 0.16666666666666663
similarity [0-1]: 0.09999999999999998


In [22]:
gs = [5, 4, 3, 2, 1, 0]

print(pearsonr(gs, sims)[0])
print(gs)
print(sims)

0.6206712050540985
[5, 4, 3, 2, 1, 0]
[0.33333333333333337, 0.33333333333333337, 0.5714285714285714, 0.33333333333333337, 0.16666666666666663, 0.09999999999999998]


In [ ]:
def morphy_tag(nltk_tag):

    if nltk_tag.startswith('J'):
        return wn.ADJ
    elif nltk_tag.startswith('V'):
        return wn.VERB
    elif nltk_tag.startswith('N'):
        return wn.NOUN
    elif nltk_tag.startswith('R'):
        return wn.ADV
    else:
        return wn.NOUN

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    w_tokenizer = WhitespaceTokenizer()
    tokenized_text = w_tokenizer.tokenize(text)
    list_tags = pos_tag(tokenized_text)
    tags = {w:morphy_tag(tag) for w,tag in list_tags}
    return [lemmatizer.lemmatize(w, tags[w]) for w in tokenized_text]

def apply_lesk_to_text(text):    
    return [lesk(text, word) for word in text]

def preprocessing(data):
    # todo: better handling of na
    data = data.fillna('')
    for column in data.columns:
        print(column)
        # remove the digits and puntuation
        data[column] = data[column].str.replace('\d+', '')
        # convert to lowercase
        data[column] = data[column].str.replace('\W+', ' ')
        # replace continuous white spaces by a single one
        data[column] = data[column].str.replace('\s+', ' ')
        # words to lower
        data[column] = data[column].str.lower()
        # lematize
        data[column] = data[column].apply(lemmatize_text)
        # data[column] = data[column].str.split()
        # desambiguate 
        data[column] = data[column].apply(apply_lesk_to_text)

    return data

In [ ]:
a = 'birdie is washing itself on the sink basine'
lemmatize_text(a)

In [ ]:
trial_df = preprocessing(trial_df)
trial_df

In [ ]:
trial_df.loc['id1','sentence0']

In [ ]:
wn.synsets('was')[0]

In [ ]:
trial_df.loc['id1','sentence1']

In [ ]:
lesk(trial_df.loc['id1','sentence0'], 'bird')

In [ ]:
def lexical_simmilarity(df):
    guess = pd.DataFrame()
    for i in df.index:
        guess.loc[i,'labels'] = 1 - jaccard_distance(set(df.loc[i,'sentence0']), set(df.loc[i,'sentence1']))
    return guess


guess_lex = lexical_simmilarity(trial_df)
guess_lex.head()

In [ ]:
print(pearsonr(trial_gs['labels'], guess_lex['labels'])[0])

In [ ]:
trial_gs

In [ ]:
trial_df.to_csv('potato.csv')